In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\conex_3\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [2]:
driver.get('https://vk.com/otchisleno')

In [3]:
search = driver.find_element_by_class_name('ui_tab_search')


In [4]:
search.click()

In [5]:
row = driver.find_element_by_id('wall_search')

In [6]:
row.clear()
row.send_keys('девушкой\n')

In [7]:
%%time
from selenium.webdriver.common.keys import Keys
import time
import numpy as np

def scroll_to_the_end(driver=driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        pause = np.abs(np.random.normal()) + 0.2
        time.sleep(pause)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
scroll_to_the_end()

Wall time: 4.04 s


In [8]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
posts = bs.findAll('div', attrs={'class': 'post_content'})

In [10]:
def filter_posts_girls(posts):
    y = lambda x: str(x).lower()
    return [x for x in posts if u'девушк' in y(x) and u'твоя' in y(x) and u'ты' not in y(x)]

In [11]:
len(posts), len(filter_posts_girls(posts))

(50, 3)

In [12]:
girls = filter_posts_girls(posts)


In [13]:
girls[0].findAll('div', attrs={'class': 'wall_post_text'})

[<div class="wall_post_text zoom_text">если бы твоя девушка была вузом то это был бы единственный вуз в который все ходили</div>]

In [14]:
%%time
row.clear()
row.send_keys('девушка \n')

time.sleep(2.3)
scroll_to_the_end()

bs = BeautifulSoup(driver.page_source, 'html.parser')

posts = bs.findAll('div', attrs={'class': 'post_content'})
def filter_posts_girls_month(posts):
    return [x for x in posts if u'девушк' in str(x).lower()]
len(filter_posts_girls_month(posts))

Wall time: 5.37 s


27

In [15]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
morph_vocab = MorphVocab()
extractor = NamesExtractor(morph_vocab)

def get_texts(posts):
    t = []
    for x in posts:
        t.extend(list(map(lambda z: z.text, x.findAll('div', attrs={'class': 'wall_post_text'}))))
    t = [x for x in t]
    return t


def save_texts(texts, name='data.txt'):
    with open(name, 'w') as f:
        f.write('\n'.join(texts))


def load_texts(name='data.txt'):
    with open(name) as f:
        return f.readlines()

In [16]:
texts = get_texts(girls)
save_texts(texts)

In [17]:
texts = load_texts()

In [25]:
driver.get('https://vk.com/search?c%5Bgroup%5D=161102237&c%5Bsection%5D=people')
sex = driver.find_element_by_xpath("//div[contains(@onclick,'radiobtn')]")
sex.click()

In [26]:
%%time
scroll_to_the_end()

Wall time: 44 s


In [27]:
bs = BeautifulSoup(driver.page_source, 'html.parser')

def get_refs2persons(page):
    return ['vk.com' + x.findAll('a')[0].get_attribute_list('href')[0] for x in page.findAll('div', attrs={'class': 'labeled name'})]

In [28]:
candidates = get_refs2persons(bs)

In [29]:
len(candidates)


1000

In [30]:
save_texts(candidates, 'candidates.txt')

In [31]:
from nltk import word_tokenize

In [32]:
driver.get('https://www.liveinternet.ru/users/4468278/post246073969/')

bs = BeautifulSoup(driver.page_source, 'html.parser')
compliments = [x for x in bs.findAll('p')]
compliments = [x for x in compliments if len(x) > 0]

In [33]:
compliments = [x.text.strip() for x in compliments]
compliments = [x for x in compliments if len(x) > 0]
compliments = [' '.join(word_tokenize(x)) for x in compliments]

In [34]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'[а-я]+|[А-Я]+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [morph.parse(w)[0].normal_form for w in tokens if morph.tag(w)[0].POS == 'ADJF']
    return filtered_words

In [35]:
tmp = []
for c in compliments:
    tmp.extend(preprocess(c))
tmp = list(set(tmp) ^ {
    'наш', 
    'свой', 
    'общий', 
    'непостоянный', 
    'беспечный', 
    'эгоистичный', 
    'сумасшедший', 
    'весь', 
    'каждый',
    'индивидуальный',
    'огнеопасный',
    'упоенный',
    'другой',
    'шипучий',
    'тот'
})

In [36]:
driver.close()

In [37]:
import torch
from torch import nn

ModuleNotFoundError: No module named 'torch'